In [ ]:
# -*- coding: utf-8 -*-
"""
Distribución de la población por sexo (auto-actualizable desde World Bank - WDI)
Muestra SIEMPRE los 5 años más recientes disponibles para el país elegido.
Estilo: Aptos Light + colores corporativos (Verde #889064 para Hombres, Naranja #ff9f18 para Mujeres).
Fuente y fecha del dato más reciente en crédito inferior derecho.
"""

import requests
import pandas as pd
import plotly.graph_objects as go
from math import ceil
import streamlit as st

# -----------------------------
# Parámetros
# -----------------------------
COUNTRY = "MEX"        # ISO3; cambia a tu gusto (ej. 'ARG', 'COL', 'USA', etc.)
LAST_N_YEARS = 5       # Siempre 5 años más recientes
IND_MALE = "SP.POP.TOTL.MA.IN"
IND_FEMALE = "SP.POP.TOTL.FE.IN"
WB_BASE = "https://api.worldbank.org/v2/country/{country}/indicator/{indicator}?format=json&per_page=20000"

# -----------------------------
# Adaptación de Estilo para Streamlit
# -----------------------------
# Fallbacks con los colores originales si no hay inyección de Streamlit
DEFAULT_FONT = "Aptos Light, Aptos, Arial, sans-serif"
# Orden de colores por defecto: [Hombres (Verde), Mujeres (Naranja)]
DEFAULT_PALETTE = ["#889064", "#ff9f18"]

# Obtener variables inyectadas de Streamlit (si existen) o usar valores por defecto
PALETTE = globals().get('active_palette', DEFAULT_PALETTE)
FONT = globals().get('active_font', DEFAULT_FONT)

# Asignar variables de estilo que se usan en la gráfica
FONT_FAMILY = FONT
# Mapeamos la paleta inyectada a los colores específicos por sexo
COL_MALE = PALETTE[0] if len(PALETTE) > 0 else DEFAULT_PALETTE[0]     # Primer color para Hombres
COL_FEMALE = PALETTE[1] if len(PALETTE) > 1 else DEFAULT_PALETTE[1]   # Segundo color para Mujeres

# -----------------------------
# Helpers
# -----------------------------
def fetch_indicator(country: str, indicator: str):
    """
    Descarga un indicador WDI para un país.
    Regresa (df con columnas [year,value], lastupdated_str).
    """
    url = WB_BASE.format(country=country, indicator=indicator)
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    js = r.json()
    meta = js[0] if isinstance(js, list) and len(js) == 2 else {}
    lastupdated = meta.get("lastupdated")  # p.ej. '2025-07-02'
    rows = js[1] if isinstance(js, list) and len(js) == 2 else []
    df = pd.DataFrame(rows)
    if df.empty:
        return pd.DataFrame(columns=["year", "value"]), lastupdated
    df = df[["date", "value"]].dropna()
    df["year"] = df["date"].astype(int)
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["value"]).loc[:, ["year", "value"]]
    return df.sort_values("year").reset_index(drop=True), lastupdated

def millions(x):
    return x / 1e6

def fmt_spanish(vals):
    """Devuelve lista de strings con coma decimal (1 decimal)."""
    return [("{:.1f}".format(v)).replace(".", ",") for v in vals]

# -----------------------------
# Descarga (Hombres y Mujeres)
# -----------------------------
df_male, upd_male = fetch_indicator(COUNTRY, IND_MALE)
df_fem,  upd_fem  = fetch_indicator(COUNTRY, IND_FEMALE)

# Intersección de años y selección de los N más recientes
common_years = sorted(set(df_male.year) & set(df_fem.year))
target_years = common_years[-LAST_N_YEARS:]
if not target_years:
    raise ValueError("No hay años en común entre los indicadores para el país seleccionado.")

df = (
    pd.DataFrame({"year": target_years})
    .merge(df_male.rename(columns={"value":"male"}), on="year", how="left")
    .merge(df_fem.rename(columns={"value":"female"}), on="year", how="left")
    .sort_values("year")
)

# A millones
df["male_m"]   = df["male"].apply(millions)
df["female_m"] = df["female"].apply(millions)

# Para etiquetas y margen superior (headroom)
ymax = max(df["male_m"].max(), df["female_m"].max())
ymax = ceil(ymax * 1.12)  # +12% de aire arriba

# Último año disponible y fecha de actualización más reciente reportada por el API
most_recent_year = target_years[-1]
updates = [u for u in [upd_male, upd_fem] if u]
last_update_str = max(updates) if updates else ""

# -----------------------------
# Gráfica (barras agrupadas)
# -----------------------------
fig = go.Figure()

fig.add_bar(
    name="Hombres",
    x=df["year"], y=df["male_m"],
    marker_color=COL_MALE, # <-- Color dinámico
    text=fmt_spanish(df["male_m"]),
    textposition="outside",
    textfont=dict(color="#3e403f", size=13),
    cliponaxis=False,
    hovertemplate="Año: %{x}<br>Hombres: %{y:.1f} millones<extra></extra>",
)

fig.add_bar(
    name="Mujeres",
    x=df["year"], y=df["female_m"],
    marker_color=COL_FEMALE, # <-- Color dinámico
    text=fmt_spanish(df["female_m"]),
    textposition="outside",
    textfont=dict(color="#3e403f", size=13),
    cliponaxis=False,
    hovertemplate="Año: %{x}<br>Mujeres: %{y:.1f} millones<extra></extra>",
)

# Layout estilo corporativo
fig.update_layout(
    barmode="group",
    bargap=0.45,
    bargroupgap=0.20,
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(l=60, r=40, t=90, b=130),
    font=dict(family=FONT_FAMILY, size=18, color="#5f6368"), # <-- Fuente dinámica
    title=dict(
        text="Distribución población por sexo<br><span style='font-size:13px;color:#9aa0a6'>(Millones)</span>",
        x=0.5, xanchor="center"
    ),
    xaxis=dict(
        title="",
        tickmode="array",
        tickvals=df["year"],
        ticktext=[str(y) for y in df["year"]],
        showgrid=False,
        tickfont=dict(color="#8a8f94", size=16)
    ),
    yaxis=dict(
        title="",
        showgrid=True,
        gridcolor="#e6e6e6",
        zeroline=False,
        rangemode="tozero",
        range=[0, ymax],
        dtick=10,
        tickfont=dict(color="#8a8f94", size=16)
    ),
    legend=dict(
        title="",
        orientation="h",
        yanchor="bottom", y=-0.25,
        xanchor="center", x=0.5,
        font=dict(size=14)
    ),
)

# Crédito inferior derecho (fuente + año y fecha de actualización)
credito = (
    "Fuente: World Bank – WDI (ONU‑WPP). "
    f"Último año disponible: {most_recent_year}"
    + (f" · Actualización WDI: {last_update_str}" if last_update_str else "")
)
fig.add_annotation(
    x=0.6, y=-0.25, xref="paper", yref="paper",
    text=credito,
    showarrow=False,
    xanchor="right", yanchor="top",
    font=dict(family=FONT_FAMILY, size=12, color="#7a7a7a") # <-- Fuente dinámica
)

# -----------------------------
# Visualización en Streamlit
# -----------------------------
st.plotly_chart(fig, use_container_width=True)

# -----------------------------
# Tabla de Datos (Visible)
# -----------------------------
st.markdown("**Datos detallados (Millones)**")

# Preparamos el DataFrame para la tabla:
# Seleccionamos las columnas calculadas en millones y el año.
# Ordenamos descendente para mostrar lo más reciente arriba.
df_table = df[['year', 'male_m', 'female_m']].copy().sort_values('year', ascending=False)

st.dataframe(
    df_table,
    use_container_width=True,
    hide_index=True,
    column_config={
        "year": st.column_config.TextColumn("Año"),
        "male_m": st.column_config.NumberColumn(
            "Hombres (Millones)",
            format="%.2f M"  # Formato: 2 decimales + M
        ),
        "female_m": st.column_config.NumberColumn(
            "Mujeres (Millones)",
            format="%.2f M"
        )
    }
)